In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from datetime import datetime


In [2]:
def create_dataset(user_id, name):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    cap = cv2.VideoCapture(0)
    count = 0
    save_path = f"dataset/{name}_{user_id}"
    os.makedirs(save_path, exist_ok=True)
    
    while count < 50:  # Capture 50 samples per user
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            file_path = os.path.join(save_path, f"{count}.jpg")
            cv2.imwrite(file_path, face)
            count += 1
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        cv2.imshow("Capturing Face", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or count >= 50:
            break

    cap.release()
    cv2.destroyAllWindows()


In [6]:
def train_model():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    face_data = []
    labels = []
    label_map = {}

    dataset_path = "dataset"
    label_id = 0

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if not os.path.isdir(folder_path):
            continue  # Skip files that are not directories
        
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            print(f"Loading image from {image_path}")  # Check the path
            
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            if image is None:
                print(f"Error: Image not found at {image_path}")
                continue  # Skip invalid images
            
            face_data.append(np.array(image, dtype="uint8"))
            labels.append(label_id)
        
        label_map[label_id] = folder
        label_id += 1

    recognizer.train(face_data, np.array(labels))
    recognizer.save("face_recognizer.yml")
    with open("label_map.npy", "wb") as f:
        np.save(f, label_map)


In [7]:
def mark_attendance(name):
    attendance_file = "attendance.csv"
    now = datetime.now()
    time = now.strftime("%H:%M:%S")
    date = now.strftime("%Y-%m-%d")
    attendance_data = []

    if os.path.exists(attendance_file):
        attendance_data = pd.read_csv(attendance_file).to_dict("records")
    
    # Avoid duplicate marking
    for entry in attendance_data:
        if entry["Name"] == name and entry["Date"] == date:
            return

    attendance_data.append({"Name": name, "Date": date, "Time": time})
    pd.DataFrame(attendance_data).to_csv(attendance_file, index=False)

def recognize_and_mark():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("face_recognizer.yml")

    with open("label_map.npy", "rb") as f:
        label_map = np.load(f, allow_pickle=True).item()

    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            label, confidence = recognizer.predict(face)
            if confidence < 50:  # Confidence threshold
                name = label_map[label]
                mark_attendance(name)
                cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Unknown", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [8]:
test_image_path = "dataset/Jaideep_001/0.jpg"
test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
if test_image is None:
    print("Failed to load test image")
else:
    print("Test image loaded successfully")


Test image loaded successfully


In [9]:
create_dataset(user_id="001", name="Jaideep")
train_model()


Loading image from dataset\Jaideep_001\.ipynb_checkpoints
Error: Image not found at dataset\Jaideep_001\.ipynb_checkpoints
Loading image from dataset\Jaideep_001\0.jpg
Loading image from dataset\Jaideep_001\1.jpg
Loading image from dataset\Jaideep_001\10.jpg
Loading image from dataset\Jaideep_001\11.jpg
Loading image from dataset\Jaideep_001\12.jpg
Loading image from dataset\Jaideep_001\13.jpg
Loading image from dataset\Jaideep_001\14.jpg
Loading image from dataset\Jaideep_001\15.jpg
Loading image from dataset\Jaideep_001\16.jpg
Loading image from dataset\Jaideep_001\17.jpg
Loading image from dataset\Jaideep_001\18.jpg
Loading image from dataset\Jaideep_001\19.jpg
Loading image from dataset\Jaideep_001\2.jpg
Loading image from dataset\Jaideep_001\20.jpg
Loading image from dataset\Jaideep_001\21.jpg
Loading image from dataset\Jaideep_001\22.jpg
Loading image from dataset\Jaideep_001\23.jpg
Loading image from dataset\Jaideep_001\24.jpg
Loading image from dataset\Jaideep_001\25.jpg
Loadin

In [10]:
recognize_and_mark()